In [1]:
from API_call import *
from functions import get_gamePks,read_gamePks
from datetime import datetime as dt
import os,re,csv
from os import walk
import statsapi as mlb
import pandas as pd
from database import *
from sqlalchemy.exc import IntegrityError

In [2]:
get_gamePks([str(x) for x in range(2008,2020)])   

gamePks=read_gamePks()

In [3]:
db = MyDatabase('sqlite',dbname='mlb.db')

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [8]:
db.Base.metadata.tables

immutabledict({})

In [5]:
Base.metadata.create_all(db.db_engine)
Base.metadata.bind=db.db_engine

In [7]:
Base.metadata.tables

immutabledict({'games': Table('games', MetaData(bind=Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)), Column('pk', Integer(), table=<games>, primary_key=True, nullable=False), Column('type', String(length=1), table=<games>), Column('doubleHeader', String(length=1), table=<games>), Column('id', String(length=150), table=<games>, primary_key=True, nullable=False), Column('gamedayType', String(length=1), table=<games>), Column('tiebreaker', String(length=1), table=<games>), Column('gameNumber', Integer(), table=<games>), Column('calendarEventID', String(length=50), table=<games>), Column('season', Integer(), table=<games>), Column('dateTime', DateTime(), table=<games>), Column('originalDate', Date(), table=<games>), Column('detailedState', String(length=12), table=<games>), Column('homeTeam_id', Integer(), table=<games>), Column('awayTeam_id', Integer(), table=<games>), Column('condition', String(length=25), table=<games>), Column('temp', Integer(), table=<games>), Column('wind', S

In [6]:
pks = gamePks['2019'][500:550]

calls = [API_call(pk) for pk in pks]
#item for sublist in list for item in sublist
records = [link for call in calls for link in call.game_player_links]

In [7]:
df = pd.DataFrame.from_records(records)

In [8]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2492 entries, 0 to 2491
Data columns (total 201 columns):
jerseyNumber                                   object
parentTeamId                                   float64
battingOrder                                   object
allPositions                                   object
person_id                                      int64
person_fullName                                object
person_link                                    object
position_code                                  object
position_name                                  object
position_type                                  object
position_abbreviation                          object
status_code                                    object
status_description                             object
gameStatus_isCurrentBatter                     bool
gameStatus_isCurrentPitcher                    bool
gameStatus_isOnBench                           bool
gameStatus_isSubstitute           

In [ ]:
df['stats_pitching_gamesPitched'].value_counts()

In [ ]:
df['gameStatus_isCurrentBatter'].value_counts()

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [ ]:
# #TEST: which columns, if any, don't have any values
# dfs = {}
# for table in db.db_engine.table_names():
#     df = pd.read_sql(f"select * from {table}",db.db_engine)
#     dfs[table]=df


# for k in dfs.keys():
#     df = dfs[k]
#     for col in df.columns:
#         if df[col].isna().sum()==df.shape[0]:
#             print(k,col)

# pks = [x[0] for x in session.query(Game.pk).all()]

# calls = [API_call(pk) for pk in pks]

# dfs_fromApi = {}
# for k in calls[0].__dict__.keys():
#     if k[0] != '_':
#         records = [x for call in calls for x in call.__dict__[k]]
#         df = pd.DataFrame.from_records(records)
#         dfs_fromApi[k] = df

In [ ]:
call = API_call(567055)

In [ ]:
liveData = call._result['liveData']

In [ ]:
box_score = liveData['boxscore']

In [ ]:
away = box_score['teams']['away']

In [ ]:
players = away['players']

In [ ]:
player_dicts = []
for k in players.keys():
    player_dicts.append(flatten_dicts(players[k]))



In [ ]:
df = pd.DataFrame.from_records(player_dicts)

In [ ]:
df.info()

In [ ]:
df['status_description'].value_counts()

In [ ]:
df.info(verbose=True)

In [ ]:
stmt = """
select 
    pk,
    count(pk) as count
from
    games
group by 
    pk
having 
    count > 1
"""

In [ ]:
db.db_engine.execute(stmt).fetchall()